In [1]:
import geopandas as gpd
import pandas as pd
import ast
import os
os.environ['PROJ_LIB'] = r'D:\Anaconda\envs\TrKG\Library\share\proj'
os.environ['GDAL_DATA'] = r'CD:\Anaconda\envs\TrKG\Library\share'
import csv
from pathlib import Path
import json
import numpy as np
from pyproj import CRS

In [ ]:
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher, Subgraph

g = Graph('bolt://localhost:7687',auth=('neo4j','your password'), name = 'trkg')

In [3]:
from shapely import wkt
from shapely.geometry import Point, LineString, MultiLineString, MultiPoint, GeometryCollection
from shapely import geometry
from shapely import ops
from shapely.ops import split, linemerge
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pickle

In [4]:
# 指定文件路径
crs_path = r"D:/paper2/result/roadcrs.txt"
# 读取投影信息
with open(crs_path, "r") as file:
    crs_text = file.read().strip()

# 将文本投影信息转换为 CRS 对象
roadcrs = CRS.from_string(crs_text)
roadcrs

<Derived Projected CRS: PROJCS["CGCS2000_3-degree_Gauss-Kruger_CM_120E",GE ...>
Name: CGCS2000_3-degree_Gauss-Kruger_CM_120E
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: China 2000
- Ellipsoid: CGCS2000
- Prime Meridian: Greenwich

In [5]:
# 从本地加载 honeycomb_cache
with open('D:/paper2/result/honeycomb_cache.pkl', 'rb') as file:
    honeycomb_cache = pickle.load(file)

In [7]:
def calculate_fid_time_summary(node_df):
    """
    使用 NumPy 优化 groupby 聚合过程
    """
    # 提前去重，获取每个 honeycomb_name 对应的固定值
    honeycomb_distances = node_df['honeycomb_name'].drop_duplicates()

    # 分组并用 NumPy 操作实现聚合
    grouped = node_df.groupby('honeycomb_name')
    start_time = grouped['datetime'].min()
    end_time = grouped['datetime'].max()
    min_name = grouped['trajectory_name'].min()
    max_name = grouped['trajectory_name'].max()
    count = max_name - min_name + 1


    # 创建 DataFrame
    fid_time_summary = pd.DataFrame({
        'honeycomb_name': start_time.index,
        'start_time': start_time.values,
        'end_time': end_time.values,
        'count': count.values,
        'min_name': min_name.values,
        'max_name': max_name.values
    })

    # 过滤 count > 1
    fid_time_summary = fid_time_summary[fid_time_summary['count'] > 1]

    # 合并 max_distance 和 min_distance
    fid_time_summary = fid_time_summary.merge(honeycomb_distances, on='honeycomb_name', how='left')

    # 计算持续时间
    fid_time_summary['duration'] = (fid_time_summary['end_time'] - fid_time_summary['start_time']).dt.total_seconds()

    # 按 start_time 排序
    fid_time_summary = fid_time_summary.sort_values(by='min_name', ignore_index=True)

    return fid_time_summary

def get_honeycomb_within(honeycomb_cache, honeycomb_name):
    """
    检查在本地缓存中是否存在指定的 honeycomb_name，并返回 True 如果找到。
    
    :param honeycomb_cache: 本地加载的 honeycomb 缓存字典
    :param honeycomb_name: 当前 honeycomb 节点的 name
    :return: True 如果存在 honeycomb_name，False 否则
    """
    # 如果在缓存中找到指定的 honeycomb_name，立即返回 True
    return honeycomb_name in honeycomb_cache

In [8]:
# 指定文件的完整路径
file_path = r'D:\paper2\result\folder_name_list.txt'
with open(file_path, 'r') as file:
    # 读取文件内容
    content = file.read()
    
    # 将字符串转换为列表
    folder_name = ast.literal_eval(content)

In [10]:
selected_files = folder_name[1156:4500]
print("selected_files第一个元素:", selected_files[0])
print("selected_files最后一个元素:", selected_files[-1])

selected_files第一个元素: 1158
selected_files最后一个元素: 4502


In [11]:
selected_files = folder_name[1156:4500]
folder_name = selected_files
print("folder_name第一个元素:", folder_name[0])
print("folder_name最后一个元素:", folder_name[-1])

folder_name第一个元素: 1158
folder_name最后一个元素: 4502


In [12]:
from tqdm import tqdm  # 导入 tqdm
import gc  # 导入垃圾回收模块

# 确保日志目录存在
log_dir = r"C:\DCLASS\myJupyter\paper2\shanghai_log\1"
os.makedirs(log_dir, exist_ok=True)
log_path = os.path.join(log_dir, "vehicle_log_2.txt")


batch_size = 250
# 计算总的进度条数：每个文件名有 8640 次迭代
total_slices = len(folder_name) * 8640

# 使用 tqdm 进度条
with tqdm(total=total_slices, desc="Processing data", unit="slice") as pbar:
    for vehicle_name in folder_name:
        
        # if vehicle_name != '1355':
        #     continue
        # if vehicle_name == '3':
        #     break
        # if vehicle_name != '1354':
        #     continue
        
        # 🔽 写入日志
        with open(log_path, "a", encoding="utf-8") as log_file:
            log_file.write(f"vehicle_name:\n {vehicle_name}\n")

        for five_times_start in range(1, 8641, batch_size):
            five_times_end = min(five_times_start + batch_size - 1, 8640)
            
            # # # 记得注释
            # if five_times_start>batch_size:
            #     break

            # 构造批次范围条件
            batch_range = ', '.join([f"'batch_{i}'" for i in range(five_times_start, five_times_end + 1)])
            
            # 参数化查询，查找 trajectory_point 的属性以及其关联的 honeycomb 节点的部分属性
            cypher = f"""
                MATCH (tp:trajectory_point)-[:located_in]->(hc:honeycomb)
                WHERE tp.vehicle = {vehicle_name} AND tp.batch IN [{batch_range}]
                AND (tp)-[:next]-()
                RETURN tp.batch AS batch, tp.geometry AS geometry, tp.name AS trajectory_name, tp.vehicle AS vehicle, tp.datetime AS datetime,
                       hc.name AS honeycomb_name
                ORDER BY tp.name
            """
            # 执行查询并传入参数
            result = g.run(cypher).data()

            # 检查是否有返回数据
            if result:
                # 将结果转换为 DataFrame
                df_result = pd.DataFrame(result)
                # 确保 datetime 列是 datetime 格式
                df_result['datetime'] = pd.to_datetime(df_result['datetime'], format="%Y-%m-%d %H:%M:%S")
                #print(f"df_result:\n {df_result}")
                
                batch_boundaries = df_result['batch'].ne(df_result['batch'].shift()).cumsum()  # 变化检测
                # 按变化边界分块

                for _, group_idx in df_result.groupby(batch_boundaries).groups.items():
                    node_df = df_result.loc[group_idx]
                    batch_name = node_df['batch'].iloc[0]  # 每个分块的 batch_name 是一致的
                
                    # trajectory_point 节点数大于 1 才进行拥堵分析
                    if len(node_df) > 1:
                        fid_time_summary = calculate_fid_time_summary(node_df)
                        # print(f"fid_time_summary:\n {fid_time_summary}")
                        # print(f"batch_name:\n {batch_name}")
                        
                        if not fid_time_summary.empty:
                            # 将 'geometry' 列中的 WKT 字符串转换为 Shapely 几何对象
                            node_df['geometry'] = node_df['geometry'].apply(wkt.loads)
                            # 转换为 GeoDataFrame
                            node_df = gpd.GeoDataFrame(node_df, geometry='geometry', crs=roadcrs)
                            # 设置 'trajectory_name' 为索引
                            node_df.set_index('trajectory_name', inplace=True)

                            # 遍历 fid_time_summary 的每一行
                            for index, row in fid_time_summary.iterrows():
                                honeycomb_name = row['honeycomb_name']
                                # print(f"\nRow {index} with FID = {honeycomb_name}")
                                duration = row['duration']
                    
                                # 检查 duration 是否为 0
                                if duration == 0:
                                    continue  # 跳过当前行
                    
    
                                # 调用 get_honeycomb_within 判断
                                if not get_honeycomb_within(honeycomb_cache, honeycomb_name):
                                    continue
                                    
                                min_name = row['min_name']
                                max_name = row['max_name']
                                # 直接使用 min_name 和 max_name 作为行索引进行切片
                                between_rows = node_df.loc[min_name:max_name]
                                # print(f"between_rows:\n {between_rows}")
            
                                # 提取几何点并构建 LineString
                                line = LineString(between_rows['geometry'].tolist())
                                
                                # 计算总长度
                                total_length = line.length
                                # print(f"Total length of the point set: {total_length}")
                                        
                                # 如果总长度小于 10 米，视为泊车状态，废弃该数据;如果总长度大于 600 米，视为错误数据，废弃该数据
                                if total_length <= 20 or total_length >= 600:
                                    continue
                    
                                # 计算平均速度 v_acc
                                v_acc = total_length / duration  # 确保单位匹配
                                
                                # 更新匹配的节点
                                cypher_update = f"""
                                MATCH (tp:trajectory_point)
                                WHERE tp.vehicle = {vehicle_name} 
                                  AND tp.batch = '{batch_name}'
                                  AND tp.name >= {min_name} AND tp.name <= {max_name}
                                SET tp.speed = {v_acc:.3f}
                                """
                                g.run(cypher_update)

                                # 输出调试信息
                                # print(f"Average speed (v_acc): {v_acc}")

                # 清除 node_df 缓存
                if 'node_df' in globals():
                    del node_df
                if 'fid_time_summary' in globals():
                    del fid_time_summary
                gc.collect()  # 手动垃圾回收

            # 清除 df_result 缓存
            if 'df_result' in globals():
                del df_result
            if 'result' in globals():
                del result
            gc.collect()  # 手动垃圾回收

            # 更新进度条
            pbar.update(five_times_end - five_times_start + 1)

    # 最后一次全局垃圾回收
    gc.collect()

Processing data: 100%|██████████████████████████████████████████████| 28892160/28892160 [109:45:42<00:00, 73.12slice/s]
